In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


In [3]:
df_fact_gold_transactions=spark.table("fact_transactions_enriched")

StatementMeta(, 404225f8-e4a5-4b99-a9d4-1badf9c844f4, 5, Finished, Available, Finished)

In [4]:
spark.sql("DROP TABLE IF EXISTS gold_fact_transactions")

StatementMeta(, 404225f8-e4a5-4b99-a9d4-1badf9c844f4, 6, Finished, Available, Finished)

DataFrame[]

In [5]:
(
    df_fact_gold_transactions
    .write
    .mode("append")
    .format("delta")
    .partitionBy("transaction_date")
    .saveAsTable("gold_fact_transactions")
)

StatementMeta(, 404225f8-e4a5-4b99-a9d4-1badf9c844f4, 7, Finished, Available, Finished)

In [3]:
df_merchant=spark.table("bronze_merchant_risk")

StatementMeta(, d2ab61a6-f924-4f65-b7c9-bcc0c46988da, 5, Finished, Available, Finished)

In [4]:
df_merchant.show()

StatementMeta(, d2ab61a6-f924-4f65-b7c9-bcc0c46988da, 6, Finished, Available, Finished)

+-----------------+----------+
|merchant_category|risk_level|
+-----------------+----------+
|        ECOMMERCE|       LOW|
|             FOOD|       LOW|
|           TRAVEL|    MEDIUM|
|    ENTERTAINMENT|       LOW|
|         GAMBLING|      HIGH|
|           CRYPTO|      HIGH|
|        ECOMMERCE|       LOW|
|             FOOD|       LOW|
|           TRAVEL|    MEDIUM|
|    ENTERTAINMENT|       LOW|
|         GAMBLING|      HIGH|
|           CRYPTO|      HIGH|
+-----------------+----------+



In [5]:
(
    df_merchant
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("gold_merchant_risk")
)

StatementMeta(, d2ab61a6-f924-4f65-b7c9-bcc0c46988da, 7, Finished, Available, Finished)

In [ ]:
df_selelct_enriched=(
    df_enriched
    .select(f.col("f.transaction_id"),
      f.col("f.transaction_date"),
      f.col("d.customer_sk"),
      f.col("f.amount"),
      f.col("d.risk_rating"),
      f.col("f.status")
)
)

In [6]:
df_customer=spark.table("dim_customer")

StatementMeta(, d2ab61a6-f924-4f65-b7c9-bcc0c46988da, 8, Finished, Available, Finished)

In [7]:
display(df_customer)

StatementMeta(, d2ab61a6-f924-4f65-b7c9-bcc0c46988da, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 635831fd-0bc6-4759-873c-bb37ebe18714)

In [10]:
from pyspark.sql import functions as f
df_select=(
    df_customer.select(
        f.col("customer_sk"),
        f.col("customer_id"),
        f.col("full_name"),
        f.col("dob"),
        f.col("account_type"),
        f.col("country"),
        f.col("created_date"),
        f.col("risk_rating"),
        f.col("is_current")
    )

)

StatementMeta(, d2ab61a6-f924-4f65-b7c9-bcc0c46988da, 12, Finished, Available, Finished)

In [11]:
(
    df_select
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("gold_dim_customers")
)

StatementMeta(, d2ab61a6-f924-4f65-b7c9-bcc0c46988da, 13, Finished, Available, Finished)

In [12]:
df_merchant_select=spark.table("silver_transactions")

StatementMeta(, d2ab61a6-f924-4f65-b7c9-bcc0c46988da, 14, Finished, Available, Finished)

In [15]:
df=(
    df_merchant_select
    .select(
        f.col("merchant"),
        f.col("merchant_category"),
    )
        .distinct()
    )


StatementMeta(, d2ab61a6-f924-4f65-b7c9-bcc0c46988da, 17, Finished, Available, Finished)

In [16]:
df.show()

StatementMeta(, d2ab61a6-f924-4f65-b7c9-bcc0c46988da, 18, Finished, Available, Finished)

+----------+-----------------+
|  merchant|merchant_category|
+----------+-----------------+
|    BET365|         GAMBLING|
|  FLIPKART|        ECOMMERCE|
|   BINANCE|           CRYPTO|
|    SWIGGY|             FOOD|
|MAKEMYTRIP|           TRAVEL|
|    AMAZON|        ECOMMERCE|
|   NETFLIX|    ENTERTAINMENT|
|      UBER|           TRAVEL|
+----------+-----------------+



In [17]:
df_add=(
    df
    .withColumn("merchant_sk",f.monotonically_increasing_id())
)

StatementMeta(, d2ab61a6-f924-4f65-b7c9-bcc0c46988da, 19, Finished, Available, Finished)

In [18]:
df_add.show()

StatementMeta(, d2ab61a6-f924-4f65-b7c9-bcc0c46988da, 20, Finished, Available, Finished)

+----------+-----------------+-----------+
|  merchant|merchant_category|merchant_sk|
+----------+-----------------+-----------+
|    BET365|         GAMBLING|          0|
|  FLIPKART|        ECOMMERCE|          1|
|   BINANCE|           CRYPTO|          2|
|    SWIGGY|             FOOD|          3|
|MAKEMYTRIP|           TRAVEL|          4|
|    AMAZON|        ECOMMERCE|          5|
|   NETFLIX|    ENTERTAINMENT|          6|
|      UBER|           TRAVEL|          7|
+----------+-----------------+-----------+



In [19]:
(
    df_add
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("gold_merchant_details")
)

StatementMeta(, d2ab61a6-f924-4f65-b7c9-bcc0c46988da, 21, Finished, Available, Finished)